In [27]:
import pandas as pd
import numpy as np
import os

In [28]:
years_dirs = os.listdir(r'./train_data/Lists ON/')

In [29]:
years_dirs

['List ON_2023',
 'List ON_2022',
 '.DS_Store',
 'List ON_2020',
 'List ON_2018',
 'List ON_2016',
 'List ON_2017',
 'List ON_2019',
 'List ON_2021']

In [30]:
def rename_unnamed(df):
    """Rename unamed columns name for Pandas DataFrame

    See https://stackoverflow.com/questions/41221079/rename-multiindex-columns-in-pandas

    Parameters
    ----------
    df : pd.DataFrame object
        Input dataframe

    Returns
    -------
    pd.DataFrame
        Output dataframe

    """
    for i, columns in enumerate(df.columns.levels):
        columns_new = columns.tolist()
        for j, row in enumerate(columns_new):
            if ("Unnamed: " in row) or ("Использование площадей объекта" in row) or ("Использование площадей здания") in row:
                columns_new[j] = np.nan
        if pd.__version__ < "0.21.0":  # https://stackoverflow.com/a/48186976/716469
            df.columns.set_levels(columns_new, level=i, inplace=True)
        else:
            df = df.rename(columns=dict(zip(columns.tolist(), columns_new)),
                           level=i)
    return df

In [31]:
arr = []
global arr

def parse(d : pd.DataFrame, file_name : str, number) -> pd.DataFrame:
    df = pd.DataFrame(rename_unnamed(d).columns.tolist(), columns=['first', 'second'])
    d.columns = df['first'].fillna(df['second'])
    inds = list(d[(d['№ п/п'].notna()) & (d['№ п/п'].str.isdigit() == False) & (d['№ п/п'] != 'Итого:')].index)
    values = list(d[(d['№ п/п'].notna()) & (d['№ п/п'].str.isdigit() == False) & (d['№ п/п'] != 'Итого:')]['№ п/п'].values)
    hash_tbl = {}
    for i in range(len(inds)-1):
        hash_tbl[values[i]] = (inds[i], inds[i+1])
    res = {}
    for key in hash_tbl.keys():
        dfch = d.iloc[hash_tbl[key][0]+1:hash_tbl[key][1]].reset_index(drop=True)
        dfch['ГЕОГРАФИЧЕСКОЕ НАЗВАНИЕ'] = [key] * dfch.shape[0]
        dfch['ПУТЬ'] = [file_name] * dfch.shape[0]
        try:
            dfch.columns = cols1
        except:
            pass
        arr.append(dfch)
    return dfch

In [32]:
dirs = [elem for elem in os.listdir(r'./train_data/Lists ON') if '.' not in elem]

In [33]:
cols1 = ['№ п/п', 'Наименование, адрес, реестровый номер объекта',
       'Год постройки', 'Дата принятия на баланс Банка России',
       'Общая площадь объекта недвижимости, \nкв. м',
       'Общая площадь здания кв. м', 'Занято службами Банка России1',
       'Не занято (не исполь\nзуется)1', 'Сдается Банком России в аренду1',
       'Аренду\nется Банком России1', 'Сдается Банком России в субаренду1',
       'Передано Банком России по договору безвозмездного пользо-вания1',
       'Передано Банку России по договору безвозмездного пользо-вания1',
       'Занято службами Банка России2', 'Не занято (не исполь\nзуется)2',
       'Сдается Банком России в аренду2', 'Аренду\nется Банком России2',
       'Сдается Банком России в субаренду2',
       'Передано Банком России по договору безвоз-мездного пользо-вания2',
       'Передано Банку России по договору безвоз-мездного пользо-вания2',
       'Балансовая стоимость, руб.', 'в том числе переоценка, руб.',
       'Сумма начисленной амортизации, руб.', 'Остаточная стоимость, руб.',
       'Техническое состояние', 'ГЕОГРАФИЧЕСКОЕ НАЗВАНИЕ', 'ПУТЬ']

In [34]:
arr = []
sum_shape = 0
for di in dirs:
    files = [elem for elem in os.listdir(r'./train_data/Lists ON' + f'/{di}') if 'DS' not in elem]
    for file in files:
        if ('2023' in file) or (('2022' in file) and ('4' in file)):
            data = pd.read_excel(r'./train_data/Lists ON' + '/' + di + '/'+ file, skiprows=5, header=[0, 1])
            sum_shape+=data.shape[0]
            parsed = parse(data, di + '/'+ file, 4)
        else:
            data = pd.read_excel(r'./train_data/Lists ON' + '/' + di + '/'+ file, skiprows=6, header=[0, 1])
            sum_shape+=data.shape[0]
            parsed = parse(data, di + '/'+ file, 4)
            parsed.columns = cols1

In [35]:
df = pd.concat(arr, axis=0).reset_index(drop=True)

In [36]:
df['Год постройки'] = np.where((df['Год постройки'] != 'Данные отсутствуют') , df['Год постройки'], None)
df['Год постройки'] = np.where((df['Год постройки'] != 'данные отсутствуют') , df['Год постройки'], None)
df['Год постройки'] = np.where((df['Год постройки'] != 'NaN') , df['Год постройки'], None)
df['ВОЗРАСТ ЗДАНИЯ'] = np.where((df['Год постройки'].notna()) & (df['Год постройки'] != 'Данные отсутствуют') | ((df['Год постройки'] != 'данные отсутствуют')), 
        2023 - df['Год постройки'].astype(float),
        None
        )

In [37]:
dupl_features = [
    'Занято службами Банка России',
    'Не занято (не исполь\nзуется)',
    'Сдается Банком России в аренду',
    'Аренду\nется Банком России',
    'Передано Банку России по договору безвозмездного пользо-вания'
]

for f in dupl_features:
    df[f] = np.where(df[f].isna(), df[f+'1'], df[f])

In [38]:
features = [
    'Наименование, адрес, реестровый номер объекта',
    'Год постройки',
    'Дата принятия на баланс Банка России',
    'Площадь объекта недвижимости \nкв. м',
    'Занято службами Банка России',
    'Не занято (не исполь\nзуется)',
    'Сдается Банком России в аренду',
    'Аренду\nется Банком России',
    'Передано Банку России по договору безвозмездного пользо-вания',
    'Передано в субаренду',
    'Балансовая стоимость, руб.',
    'в том числе переоценка, руб.',
    'Сумма начисленной амортизации, руб.',
    'Остаточная стоимость, руб.',
    'Техническое состояние',
    'ГЕОГРАФИЧЕСКОЕ НАЗВАНИЕ',
    'ПУТЬ',
    'ВОЗРАСТ ЗДАНИЯ'
]

In [39]:
df['Передано в субаренду'] = np.where(df['Передано в субаренду'].isna(), 
                                     df['Сдается Банком России в субаренду1'],
                                     df['Передано в субаренду'])
df['Занято службами Банка России'] = np.where(df['Занято службами Банка России'].isna(), df['Занято службами Банка России1'], df['Занято службами Банка России'])
df['Не занято (не исполь\nзуется)'] = np.where(df['Не занято (не исполь\nзуется)'].isna(), df['Не занято (не исполь\nзуется)1'], df['Не занято (не исполь\nзуется)'])
df['Площадь объекта недвижимости \nкв. м'] = np.where(df['Площадь объекта недвижимости \nкв. м'].isna(),
                                                     df['Общая площадь объекта недвижимости, \nкв. м'],
                                                      df['Площадь объекта недвижимости \nкв. м']
                                                     )

In [40]:
def transform_year(d):
    return d.split('/')[0][-4:]

def transform_q(d):
    return d.split('/')[1][:1]

In [41]:
df_clear = df[features]
df['ГОД'] = df_clear['ПУТЬ'].apply(transform_year)
df['КВАРТАЛ'] = df_clear['ПУТЬ'].apply(transform_q)

In [42]:
df[features + ['ГОД', 'КВАРТАЛ']]

,"Наименование, адрес, реестровый номер объекта",Год постройки,Дата принятия на баланс Банка России,Площадь объекта недвижимости \nкв. м,Занято службами Банка России,Не занято (не исполь\nзуется),Сдается Банком России в аренду,Аренду\nется Банком России,Передано Банку России по договору безвозмездного пользо-вания,Передано в субаренду,"Балансовая стоимость, руб.","в том числе переоценка, руб.","Сумма начисленной амортизации, руб.","Остаточная стоимость, руб.",Техническое состояние,ГЕОГРАФИЧЕСКОЕ НАЗВАНИЕ,ПУТЬ,ВОЗРАСТ ЗДАНИЯ,ГОД,КВАРТАЛ
0,Центральный банк Российской Федерации (Банк Ро...,1907,28.11.1935,1465.1,1465.1,NaN,NaN,NaN,NaN,NaN,3.329127e+06,949105.64,3187766.60,1.413599e+05,Удовлетворительное,Дальневосточное ГУ Банка России,List ON_2023/1 quarter 2023.xls,116.0,2023,1
1,"здание, Российская Федерация, Приморский край,...",1997,31.12.1997,1292.6,1286.1,NaN,NaN,NaN,NaN,NaN,1.306612e+07,1902471.37,3299194.31,9.766922e+06,Хорошее,Дальневосточное ГУ Банка России,List ON_2023/1 quarter 2023.xls,26.0,2023,1
2,"Центральный банк Российской Федерации, Российс...",1996,03.12.1997,3789.6,3779.0,NaN,NaN,NaN,NaN,NaN,3.545058e+07,NaN,6547461.94,2.890312e+07,Удовлетворительное,Дальневосточное ГУ Банка России,List ON_2023/1 quarter 2023.xls,27.0,2023,1
3,"помещения, Российская Федерация, Приморский кр...",None,31.03.1995,130.1,130.1,130.1,NaN,NaN,NaN,NaN,1.249596e+06,NaN,349886.87,8.997091e+05,Удовлетворительное,Дальневосточное ГУ Банка России,List ON_2023/1 quarter 2023.xls,NaN,2023,1
4,"Подземное сооружение (склад), Приморский край,...",None,NaN,94.3,94.3,NaN,NaN,NaN,NaN,NaN,1.441265e+05,84523.39,94625.60,4.950093e+04,Удовлетворительное,Дальневосточное ГУ Банка России,List ON_2023/1 quarter 2023.xls,NaN,2023,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12062,"Здание РКЦ, 689450, Чукотский автономный округ...",1987.0,09.01.2016,476.2,476.2,0.0,0.0,0.0,0.0,0.0,1.053991e+07,9526978.74,3384416.41,7.155498e+06,Хорошее,Отделение Анадырь,List ON_2021/3 quarter 2021.xls,36.0,2021,3
12063,"Помещение гаража Здания РКЦ, 689450, Чукотский...",1987.0,09.01.2016,70.1,70.1,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,Хорошее,Отделение Анадырь,List ON_2021/3 quarter 2021.xls,36.0,2021,3
12064,"пункт резервного энергоснабжения, 689100, Чуко...",1997.0,09.01.2016,46.1,0.0,46.1,0.0,0.0,0.0,0.0,3.167024e+06,78402.86,911867.11,2.255157e+06,Хорошее,Отделение Анадырь,List ON_2021/3 quarter 2021.xls,26.0,2021,3
12065,Теплая стоянка пункта резервного энергоснабжен...,1997.0,09.01.2016,43.9,0.0,43.9,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,Хорошее,Отделение Анадырь,List ON_2021/3 quarter 2021.xls,26.0,2021,3
